In [1]:
import requests
import re
import json
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
import os
os.chdir('ruc_程序设计大作业/code/data')

In [2]:
class FundHistoryData:
    """
    一个类，用于从东方财富API获取指定基金的历史净值数据

    属性:
        fund_id (int): 基金代码
        max_pages (int): 要获取的最大页数
        output_file (str): 保存数据的CSV文件名
    """

    def __init__(self, fund_id: int, max_pages: int = None):
        """
        初始化FundHistoryData类
        :param fund_id: 基金代码
        :param max_pages: 获取数据的最大页数
        :param output_file: 保存数据的文件名
        """
        self.api_url = 'http://api.fund.eastmoney.com/f10/lsjz'
        self.fund_id = fund_id
        self.session = requests.Session() # 知识点：创建会话对象，保持会话状态
        self.max_pages = max_pages
        self.headers = {
            'Host': 'api.fund.eastmoney.com',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
            'Referer': f'http://fundf10.eastmoney.com/jjjz_{self.fund_id}.html',
        } # 知识点：设置请求头，模拟浏览器访问，以避免被服务器拒绝

    def get_fund_details(self):
        """
        获取并显示基金的基本信息

        :return: 成功获取信息返回True，否则返回False
        """
        search_url = 'https://fundsuggest.eastmoney.com/FundSearch/api/FundSearchAPI.ashx'
        params = {
            "callback": "jQuery18309325043269513131_1618730779404",
            "m": 1,
            "key": self.fund_id,
        }
        response = self.session.get(search_url, params=params) # 知识点：使用会话对象发送请求
        try: # 知识点：异常处理，try-except
            content = self._parse_content(response.text) # 知识点：调用静态方法
            fund_info = content['Datas'][0]
            print(f"{'*' * 30}")
            print(f"* 爬取对象: {fund_info['NAME']} *")
            print(f"{'*' * 30}")
            return True
        except (KeyError, IndexError, TypeError):
            print(f'无法获取基金代码为 {self.fund_id} 的详细信息，请检查代码。')
            return False

    @staticmethod # (*)知识点：设置静态方法staticmethod，不需要访问实例属性
    def _parse_content(content: str) -> dict:
        """
        解析API返回的原始内容字符串为JSON对象

        :param content: API响应的原始内容字符串
        :return: 解析后的JSON对象

        """
        pattern = re.compile(r'jQuery.+?\((.*)\)') # (*)知识点：使用re模块借助正则表达式解析字符串
        data = json.loads(pattern.findall(content)[0]) # (*)知识点：使用json模块解析JSON字符串
        return data

    def _fetch_page(self, page_number: int) -> dict:
        """
        获取指定页码的数据。

        :param page_number: 要获取数据的页码。
        :return: 指定页码的数据，格式为JSON。
        """
        params = {
            'callback': 'jQuery18308909743577296265_1618718938738',
            'fundCode': self.fund_id,
            'pageIndex': page_number,
            'pageSize': 20,
        }
        response = self.session.get(url=self.api_url, headers=self.headers, params=params)
        content = self._parse_content(response.text)
        return content

    def _page_iterator(self):
        """
        根据最大页码数进行分页迭代。

        :yield: 每页的数据，格式为JSON。
        """
        for page_number in range(1, self.max_pages + 1):
            yield self._fetch_page(page_number) # 知识点：使用生成器函数yield返回数据

    def _fetch_all_data(self):
        """
        自动迭代所有可用页码以获取所有数据。

        :yield: 每页的数据，格式为JSON。
        知识点：利用生成器函数实现了数据的逐页获取和迭代。
        """
        total_records = float('inf')
        page_number = 1
        while (page_number - 1) * 20 < total_records:
            page_data = self._fetch_page(page_number)
            total_records = page_data.get('TotalCount', 0)
            page_number += 1
            yield page_data

    def fetch_data(self) -> pd.DataFrame:
        """
        获取基金数据并保存到CSV文件。
        :return: 包含数据的pandas DataFrame。
        """
        if self.get_fund_details():
            data_frame = pd.DataFrame()
            if self.max_pages:
                for page_data in tqdm(self._page_iterator()):
                    page_df = pd.DataFrame(page_data['Data']['LSJZList'])[['FSRQ', 'DWJZ']]
                    page_df.columns = ['datetime', 'close']
                    data_frame = data_frame.append(page_df, ignore_index=True)
            else:
                for page_data in tqdm(self._fetch_all_data()):
                    page_df = pd.DataFrame(page_data['Data']['LSJZList'])[['FSRQ', 'DWJZ']]
                    page_df.columns = ['datetime', 'close']
                    data_frame = data_frame.append(page_df, ignore_index=True)
            data_frame.sort_values(by='datetime', inplace=True)
            data_frame.reset_index(drop=True, inplace=True)
            return data_frame

In [3]:
fund_id = 510050
fetcher = FundHistoryData(fund_id)
data = fetcher.fetch_data()
data.to_csv(f'./raw/crawl_index_1d_SH510050.csv', index=False)

******************************
* 爬取对象: 华夏上证50ETF *
******************************


237it [00:19, 11.99it/s]
